# Sharing access to a server

This notebook executes some javascript in the browser, using the user's OAuth token.

This code would normally reside in a jupyterlab extension.
The notebook serves only for demonstration purposes.

First, collect some configuration from the page, so we can talk to the JupyterHub API:

In [1]:
%%javascript
// define some globals to share

var configElement = document.getElementById("jupyter-config-data");
var jupyterConfig = JSON.parse(configElement.innerHTML);

window.token = jupyterConfig.token;
window.hubOrigin = `${document.location.protocol}//${jupyterConfig.hubHost || window.location.host}`
window.hubUrl = `${hubOrigin}${jupyterConfig.hubPrefix}`;
window.shareCodesUrl = `${hubUrl}api/share-codes/${jupyterConfig.hubServerUser}/${jupyterConfig.hubServerName}`
window.sharesUrl = `${hubUrl}api/shares/${jupyterConfig.hubServerUser}/${jupyterConfig.hubServerName}`
console.log(shareCodesUrl);

// utility function to make API requests and parse errors
window.apiRequest = async function (url, options) {
  var element = options.element;
  var okStatus = options.ok || 200;
  var resp = await fetch(url, {headers: {Authorization: `Bearer ${token}`}, method: options.method || 'GET'});
  var replyText = await resp.text();
  var replyJSON = {};
  if (replyText.length) {
     replyJSON = JSON.parse(replyText);
  }
  
  if (resp.status != okStatus) {
    var p = document.createElement('p');
    p.innerText = `Error ${resp.status}: ${replyJSON.message}`;
    element.appendChild(p);
    return;
  }
  return replyJSON;
}

// `element` is a special variable for the current cell's output area
element.innerText = `API URL for sharing codes is: ${shareCodesUrl}`;

<IPython.core.display.Javascript object>

Next, we can request a share code with

```
POST $hub/api/share-codes/$user/$server
```

The URL for _accepting_ a sharing invitation code is `/hub/accept-share?code=abc123...`:

In [2]:
%%javascript

(async function f() {
  var shareCode = await apiRequest(shareCodesUrl, {method: 'POST', element: element});

  // laziest way to display
  var shareCodeUrl = `${hubOrigin}${shareCode.accept_url}`
  var a = document.createElement('a');
  a.href = shareCodeUrl;
  a.innerText = shareCodeUrl;
  var p = document.createElement(p);
  p.append("Share this URL to grant access to this server: ");
  p.appendChild(a);
  element.appendChild(p);
})();



<IPython.core.display.Javascript object>

Share this URL to grant access to your server (e.g. visit the URL in a private window and login as the user `shared-with`).

After our code has been used, we can see who has access to this server:

In [4]:
%%javascript

(async function f() {

  var shares = await apiRequest(sharesUrl, {element: element});

  var list = document.createElement('ul');
  for (var share of shares.items) {
    var p = document.createElement('li');
    p.append(`${share.kind} ${share[share.kind].name} has access: `)
    var scopes = document.createElement('tt');
    scopes.innerText = share.scopes.join(',');
    p.appendChild(scopes);
    list.append(p);
  }
  var p = document.createElement('p');
  p.innerText = `Shared with ${shares.items.length} users:`;
  element.appendChild(p);
  element.appendChild(list);
  return;
})();


<IPython.core.display.Javascript object>

We could also use this info to revoke permissions, or share with individuals by name.

We can also review outstanding sharing _codes_:

In [5]:
%%javascript

(async function f() {
  var shareCodes = await apiRequest(shareCodesUrl, {element: element});
  var p = document.createElement('pre');
  p.innerText = JSON.stringify(shareCodes.items, null, ' ');
  element.appendChild(p);
})();


<IPython.core.display.Javascript object>

And finally, when we're done, we can revoke the codes, at which point nobody _new_ can use the code to gain access to this server,
but anyone who has accepted the code will still have access:

In [6]:
%%javascript

(async function f() {
  await apiRequest(shareCodesUrl, {method: 'DELETE', element: element, ok: 204});
  var p = document.createElement('p');
  p.innerText = `Deleted all share codes`;
  element.appendChild(p);  
})();

<IPython.core.display.Javascript object>

Or even revoke all shared access, so anyone who may have used the code no longer has any access:

In [7]:
%%javascript

(async function f() {
  var resp = await apiRequest(sharesUrl, {method: 'DELETE', element: element, ok: 204});
  var p = document.createElement('p');
  p.innerText = `Deleted all shared access`;
  element.appendChild(p);  
})();

<IPython.core.display.Javascript object>